In [32]:
import torch
import numpy as np 
import pandas as pd 
from transformers import BertTokenizer, BertModel, BertForMaskedLM, AutoTokenizer, AutoModelForMaskedLM,AutoModel
from scipy.spatial.distance import cosine 
import tokenizers 
import pandas as pd 
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
from nltk.corpus import stopwords
import snowballstemmer 
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import snowballstemmer
import numpy
import os 
import re
import json
import pickle 

In [72]:
model = AutoModelForMaskedLM.from_pretrained("Shushant/nepaliBERT", output_hidden_states = True, return_dict = True, output_attentions = True)

In [73]:
tokenizers = AutoTokenizer.from_pretrained("Shushant/nepaliBERT")

In [ ]:
tokenizers.tokenize("के मौजुदा लोकतान्त्रिक व्यवस्था राज्य पुनःसंरचनासँग जोडिएका हिजोका सवालहरूलाई यथास्थितिमा छोडेर सबल होला?")

In [ ]:
# text = 'अनि तेस्रो चिन्ता मौसम परिवर्तनले हिमशिखरहरूमा परेका आघातसँगसँगै सिमानावारिपारि नदीले ल्याएका प्रकोपहरू कसरी सम्हाल्ने'
# marked_text = " [CLS] "+text+" [SEP] "
# tokenized_text = tokenizer.tokenize(marked_text)
# indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# segments_ids = [1] * len(indexed_tokens)

# tokens_tensors = torch.tensor([indexed_tokens])
# segments_tensors = torch.tensor([segments_ids])

In [ ]:
# with torch.no_grad():
#     outputs = model(tokens_tensors, segments_tensors)
#     hidden_states = outputs.hidden_states
# #     print(hidden_states[-1])
#     token_embeddings = hidden_states[-1]
    
#     token_embeddings = torch.squeeze(token_embeddings, dim = 0)
    
#     list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]
#     print(list_token_embeddings)

In [ ]:
nepali_stemmer = snowballstemmer.NepaliStemmer()

In [ ]:
texts = ['तर','दुधमा तर बसेन|','तिम्रो घर आउन मन लाग्छ तर अल्छि लाग्छ']

In [ ]:
def bert_text_preparation(text, tokenizer ):
    """Preparing input for BERT"""
    
    marked_text = " [CLS] " + text + " [SEP] "
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(indexed_tokens) 
    
    # Convert inputs to Pytorch tensors
    tokens_tensors = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    return tokenized_text, tokens_tensors, segments_tensors

In [ ]:
def get_bert_embeddings(tokens_tensor, segments_tensors, model):
    # Gradient claculation id disabled 
    # Model is in inference mode
    
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        # removing the first hidden state
        # the first state is the input state 
        hidden_states = outputs.hidden_states
    
    # Getting embeddings from final Bert Layer
    tokens_embeddings = hidden_states[-1]
    # Collasping the tensor into 1-dimension 
    tokens_embeddings = torch.squeeze(tokens_embeddings, dim = 0)
    # Converting torchtensors to lists 
    list_token_embeddings = [token_embed.tolist() for token_embed in tokens_embeddings]
    
    return list_token_embeddings 

In [ ]:
target_word_embeddings = []

for text in texts:
    tokenized_text, tokens_tensors, segments_tensors = bert_text_preparation(text, tokenizers)
    list_token_embeddings = get_bert_embeddings(tokens_tensors, segments_tensors, model)
#     print(len(list_token_embeddings))
    ## list_token_embeddings has embeddings of the given words
    word_index = tokenized_text.index('तर')
    word_embedding = list_token_embeddings[word_index]
#     print(word_embedding)
    target_word_embeddings.append(word_embedding)

In [ ]:
target_word_embeddings[0] == target_word_embeddings[1]

In [ ]:
len(target_word_embeddings)

In [39]:
# list_of_distances = []
# for text1, embed1 in zip(texts, target_word_embeddings):
#     for text2, embed2 in zip(texts, target_word_embeddings):
#         cos_dist = 1 - cosine(embed1,embed2)
#         list_of_distances.append([text1, text2, cos_dist])


# distances_df = pd.DataFrame(list_of_distances, columns = ['text1','text2','distance'])


In [ ]:
# df = pd.read_csv("finalData.csv")
df = pd.read_csv('collected_labeled_data.csv')

In [ ]:
df['label'].unique()

In [25]:
# df.to_csv('collected_labeled_data.csv',index = False)

In [26]:
# train_X, test_X = train_test_split(df, test_size = 0.2)

In [27]:
# train_X.to_csv('train.csv',index = False)
# test_X.to_csv('test.csv',index = False)

In [28]:
# def check_len(text):
#     txt = text.split(' ')[:20]
#     return ' '.join(txt)

In [29]:
# df['text'] = df['text'].apply(check_len)

In [30]:
# def get_word_embeddings(text):
#     tokenizer = tokenizers
#     tokenized_text, tokens_tensors, segments_tensors = bert_text_preparation(text, tokenizer)
#     list_token_embeddings = get_bert_embeddings(tokens_tensors, segments_tensors, model)
#     ## list_token_embeddings has embeddings of the given words
#     return list_token_embeddings

In [40]:
stopwords= stopwords.words("nepali")

In [41]:
words = ['अक्सर','आदि','कसरी','अन्तर्गत','अर्थात','अर्थात्','अलग','आयो','उदाहरण','एकदम','राम्रो','बिरुद्ध','बिशेष','नराम्रो']

In [42]:
stopwords = list(set(stopwords).difference(set(words)))

In [43]:
def remove_emojis(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f" # dingbats
        u"\u3030"
    "]+", re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    return text

In [63]:
def clean_text(text):
    text = remove_emojis(text)
    text = text.split(' ')
    clean_text_list = []
    for word in text:
        if word not in stopwords:
            clean_text_list.append(word)
    clean_text = ' '.join(clean_text_list)
    stem_words = nepali_stemmer.stemWords(clean_text.split())
#     stem_text = ' '.join(stem_words)
#     txt = re.sub(r"[|a-zA-z.'#0-9@,:?'\u200b\u200c\u200d!/&~-]",'',stem_text)
    return ' '.join([i for i in stem_words])

In [64]:
clean_text("घाम जति लग्यो हामीलाई तेती राम्रो हुन्छ apple ")

'घाम जति लग् हामी तेती राम्रो apple'

In [65]:
nepali_stemmer.stemWords("घाम जति लग्यो हामीलाई तेती राम्रो हुन्छ apple ".split())

['घाम', 'जति', 'लग्', 'हामी', 'तेती', 'राम्रो', '', 'apple']

In [66]:
df['text'] = df['text'].apply(clean_text)


In [67]:
df.head()

,text,label
0,बजार जसरी ट्रेन्ड चेन्ज गर् हेर् प्रोफिट बूकिङ...,2
1,1000 अंक घट नेप्से 200 अंक बढ् ठूलो कुरो होइन ...,1
2,होइन सानि बैंक bonus घोसणा २ महिना (book clos...,2
3,"खैँ MBJC कित्ता रू,10/- बढेर आज रू,1100/- 10क...",2
4,राम्रो,1


In [74]:
def get_bert_embedding_sentence(input_sentence):
    md = model
    tokenizer = tokenizers
    marked_text = " [CLS] " + input_sentence + " [SEP] "
    tokenized_text = tokenizer.tokenize(marked_text)

    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(indexed_tokens) 
    
    # Convert inputs to Pytorch tensors
    tokens_tensors = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    with torch.no_grad():
        outputs = md(tokens_tensors, segments_tensors)
        # removing the first hidden state
        # the first state is the input state 

        hidden_states = outputs.hidden_states
#         print(hidden_states[-2])
        # second_hidden_states = outputs[2]
    # hidden_states has shape [13 x 1 x 22 x 768]

    # token_vecs is a tensor with shape [22 x 768]
#     token_vecs = hidden_states[-2][0]
    # get last four layers
#     last_four_layers = [hidden_states[i] for i in (-1,-2, -3,-4)]


    # cast layers to a tuple and concatenate over the last dimension
#     cat_hidden_states = torch.cat(tuple(last_four_layers), dim=-1)
#     print(cat_hidden_states.shape)
    token_vecs = hidden_states[-2][0]

    # take the mean of the concatenated vector over the token dimension
#     sentence_embedding = torch.mean(cat_hidden_states, dim=0).squeeze()

    # Calculate the average of all 22 token vectors.
    sentence_embedding = torch.mean(token_vecs, dim=0)
#     sentence_embedding = torch.mean(token_vecs, dim=1)
    return sentence_embedding.numpy()

In [58]:
# get_bert_embedding_sentence("नेपाल को ससकृती ध्वस्त पार्ने योजना")

In [69]:
df=df.drop(df[df['label']==2].index)

In [70]:
df.dropna(inplace = True)

In [75]:
df['word_embeddings'] = df['text'].apply(get_bert_embedding_sentence)

In [76]:
df.shape

(6056, 3)

In [77]:
df.head()

,text,label,word_embeddings
1,1000 अंक घट नेप्से 200 अंक बढ् ठूलो कुरो होइन ...,1,"[-0.2517209, 0.80447733, -0.30090085, 0.363934..."
4,राम्रो,1,"[-0.4275645, 0.90052205, -0.6469192, 0.3758416..."
6,जानकारी धन्यवाद रामहरी ब्रदर,1,"[0.24045938, 0.72639877, -0.11193645, 0.146293..."
18,"भारत-मधेस नेपाल-चीन सम्बन्ध विग्रन्छ, मधेसी ने...",0,"[0.15390012, 0.67477095, -0.1543702, -0.212426..."
25,"लेखनाथ न्यौपा खुलासा,महाधिबेशन एमसीसी गर् जुत्...",0,"[-0.07738958, 1.039313, -0.1071973, -0.0086015..."


In [78]:
# df.to_csv('embedding_data.csv',index = False)

In [79]:
X,y = df['word_embeddings'], df['label']

In [80]:
# scaler = StandardScaler()
# pca = PCA(n_components = 768)

In [81]:
# scaled_X = scaler.fit_transform(X.tolist())
# pca_X = pca.fit_transform(scaled_X)

In [82]:
train_X, test_X, train_y, test_y = train_test_split(X,y, test_size = 0.2, random_state = 420)

In [83]:
svc = SVC()

In [84]:
# train_X = [i[0] for i in train_X]
# test_X = [i[0] for i in test_X]

In [85]:
# train_X[0][0].shape

In [86]:
svc.fit(train_X.tolist(), train_y)
#svc.fit(train_X, train_y)

SVC()

In [87]:
svc_pred = svc.predict(test_X.tolist())
# svc_pred = svc.predict(test_X)

In [88]:
print(confusion_matrix(test_y, svc_pred))

[[424  91]
 [ 79 618]]


In [89]:
print(classification_report(test_y, svc_pred))

              precision    recall  f1-score   support

           0       0.84      0.82      0.83       515
           1       0.87      0.89      0.88       697

    accuracy                           0.86      1212
   macro avg       0.86      0.85      0.86      1212
weighted avg       0.86      0.86      0.86      1212



In [90]:
accuracy_score(test_y, svc_pred)

0.8597359735973598

In [91]:
f1_score(test_y, svc_pred)

0.8790896159317211

In [92]:
sent = "नराम्रो"
predicted_label = svc.predict(np.array(get_bert_embedding_sentence(sent).tolist()).reshape(1,-1))[0]
if predicted_label == 0:
    print(f'{sent} is negative sentiment')
else:
    print(f'{sent} is positive sentiment')

नराम्रो is negative sentiment


In [24]:
pickle.dump(svc, open('scv_sentiment','wb'))

In [18]:

# pickle.dump(svc,open('svc_sentiment','wb'))

In [22]:
svc_sentiment = pickle.load(open('svc_sentiment','rb'))

In [113]:
svc.predict(np.array(get_bert_embedding_sentence("देश बिग्रियो").tolist()).reshape(1,-1))[0]

0